In [1]:
%run -n main.py
lines = read_lines(DOTENV_PATH)
pairs = parse_dotenv(lines)
os.environ.update(pairs)
%run -n main.py

# tasks

## terra

In [ ]:
path = 'data/rsg/TERRa/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == 'entailment'], 50)
    + random.sample([_ for _ in items if _['label'] == 'not_entailment'], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/terra.jsonl', lines)

## danetqa

In [ ]:
path = 'data/rsg/DaNetQA/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] is True], 50)
    + random.sample([_ for _ in items if _['label'] is False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/danetqa.jsonl', lines)

## parus

In [ ]:
path = 'data/rsg/PARus/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['question'] == 'effect'], 48)
    + random.sample([_ for _ in items if _['question'] == 'cause'], 52)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/parus.jsonl', lines)

## rwsd

In [55]:
path = 'data/rsg/RWSD/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [67]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == True], 50)
    + random.sample([_ for _ in items if _['label'] == False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [69]:
lines = format_jsonl(items)
write_lines('tasks/rwsd.jsonl', lines)

## russe

In [70]:
path = 'data/rsg/RUSSE/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [77]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == True], 50)
    + random.sample([_ for _ in items if _['label'] == False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [78]:
lines = format_jsonl(items)
write_lines('tasks/russe.jsonl', lines)

## muserc

In [ ]:
path = 'data/rsg/MuSeRC//val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

# eval

In [137]:
%run -n main.py
task = RUSSE
eval = '11_turbo_russe'

In [138]:
%run -n main.py
lines = read_lines(f'tasks/{task}.jsonl')
task_items = list(parse_jsonl(lines))

In [139]:
task_item

{'text': 'На вывесках над дверями лавочек были изображения, показывающие, какого рода работы выполняют внутри. Хотя все больше людей учились читать, каждый ремесленник все еще использовал такие знаки, не желая потерять возможного клиента только потому, что он оказался неграмотным.',
 'target': {'span2_index': 34,
  'span1_index': 30,
  'span1_text': 'клиента',
  'span2_text': 'он'},
 'label': True,
 'id': 50}

In [144]:
%run -n main.py
task_item = random.choice(task_items)
prompt = TASK_PROMPTS[task](task_item)
print(prompt)

Does Word have the same meaning in Sentence A and Sentence B?
Keep it short, respond Yes or No.
---
Word: дорожка
Sentence A: Бурые ковровые дорожки заглушали шаги
Sentence B: Приятели решили выпить на дорожку в местном баре
Answer: No
---
Word: защита
Sentence A: Как изменится защита Динамо в новом сезоне?
Sentence B: Обе партии протекали одинаково: в обеих была разыграна французская защита
Answer: Yes
---
Word: жилец
Sentence A: По домам ходили агитаторы. Уговаривали жильцов проголосовать как можно раньше
Sentence B: Передай вашему жильцу, что он должен непременно учиться
Answer: 


In [145]:
%run -n main.py
response = openai_chat_completions(prompt, model=GPT_35_TURBO_0301, stop='---')
print(response)

Yes


In [152]:
path = Path(f'evals/{eval}.jsonl')
eval_items = []
if path.exists():
    lines = read_lines(path)
    eval_items.extend(parse_jsonl(lines))
len(eval_items)

11

In [153]:
%run -n main.py
cache_ids = {_['id'] for _ in eval_items}
nocache_task_items = [_ for _ in task_items if _['id'] not in cache_ids]

for task_item in log_progress(nocache_task_items):
    prompt = TASK_PROMPTS[task](task_item)

    response = openai_chat_completions(prompt, model=GPT_35_TURBO_0301, stop='---')
    sleep(2)

    # response = cohere_generate(prompt, end_sequences=['---'])
    # sleep(12)

    eval_items.append({
        'id': task_item['id'],
        'response': response
    })


100%|██████████| 89/89 [04:46<00:00,  3.22s/it]


In [148]:
eval_items

[{'id': 2699, 'response': 'Yes'},
 {'id': 3214, 'response': 'No'},
 {'id': 6339, 'response': 'No'},
 {'id': 1064, 'response': 'Yes'},
 {'id': 2673, 'response': 'No'},
 {'id': 6212, 'response': 'Yes'},
 {'id': 1104, 'response': 'Yes'},
 {'id': 6190, 'response': 'No'},
 {'id': 1196, 'response': 'Yes'},
 {'id': 3420, 'response': 'Yes'},
 {'id': 7150, 'response': 'No'}]

In [149]:
[_['label'] for _ in task_items[:10]]

[False, False, False, True, True, True, True, False, False, False]

In [154]:
lines = format_jsonl(eval_items)
write_lines(f'evals/{eval}.jsonl', lines)

# score

In [155]:
%run -n main.py
model_task_evals = [
    (TEXT_DAVINCHI_003, TERRA, '01_davinci_terra'),
    (TEXT_DAVINCHI_003, DANETQA, '02_davinci_danetqa'),
    (TEXT_DAVINCHI_003, PARUS, '03_davinci_parus'),
    (GPT_35_TURBO_0301, PARUS, '04_turbo_parus'),
    (GPT_35_TURBO_0301, DANETQA, '05_turbo_danetqa'),
    (GPT_35_TURBO_0301, TERRA, '06_turbo_terra'),
    (COHERE_XLARGE, PARUS, '07_cohere_parus'),
    (COHERE_XLARGE, DANETQA, '08_cohere_danetqa'),
    (COHERE_XLARGE, TERRA, '09_cohere_terra'),
    (GPT_35_TURBO_0301, RWSD, '10_turbo_rwsd'),
    (GPT_35_TURBO_0301, RUSSE, '11_turbo_russe'),
]
data = []
for model, task, eval in model_task_evals:
    lines = read_lines(f'tasks/{task}.jsonl')
    id_targets = {
        _['id']: _['label']
        for _ in parse_jsonl(lines)
    }
    
    lines = read_lines(f'evals/{eval}.jsonl')
    norm_response = NORM_RESPONSES[task]
    id_preds = {
        _['id']: norm_response(_['response'])
        for _ in parse_jsonl(lines)
    }
    score = acc_score(id_targets, id_preds)
    data.append((model, task, score))
data

[('text-davinci-003', 'terra', (0.91, 0)),
 ('text-davinci-003', 'danetqa', (0.79, 0)),
 ('text-davinci-003', 'parus', (0.93, 0)),
 ('gpt-3.5-turbo-0301', 'parus', (0.9130434782608695, 8)),
 ('gpt-3.5-turbo-0301', 'danetqa', (0.8350515463917526, 3)),
 ('gpt-3.5-turbo-0301', 'terra', (0.86, 0)),
 ('xlarge', 'parus', (0.54, 0)),
 ('xlarge', 'danetqa', (0.5, 0)),
 ('xlarge', 'terra', (0.72, 0)),
 ('gpt-3.5-turbo-0301', 'rwsd', (0.5656565656565656, 1)),
 ('gpt-3.5-turbo-0301', 'russe', (0.65, 0))]

# explore

In [131]:
# lines = read_lines(f'tasks/{task}.jsonl')
# id_task_items = {
#     _['id']: _
#     for _ in parse_jsonl(lines)
# }

# for id, task_item in id_task_items.items():
#     target = id_targets[id]
#     pred = id_preds[id]
#     if target == pred:
#         continue
        
#     display(task_item)
#     display(target)
#     display(pred)